In [1]:
# import the opencv library
import cv2
from viewer import Viewer

In [2]:
viewer = Viewer()

In [3]:
import torch
import os
import numpy as np

from models import UNET
import math

In [4]:
from data_handlers.augmentations import get_transforms_for_predict

In [5]:
def get_model(i):
    checkpoint_dir = '/mnt/heap/My folder/tune_reports/Unet/find_hyperparameters_113ba_00000_0_2022-03-29_22-29-48/checkpoint_0000'
    s = '0' + str(i) if i < 10 else str(i)
    checkpoint_dir += s
    
    torch_model = UNET(in_channels=3, out_channels=3)
    checkpoint = os.path.join(checkpoint_dir, "checkpoint")
    model_state, optimizer_state = torch.load(checkpoint)
    torch_model.load_state_dict(model_state)
    torch_model = torch_model.eval() 
    torch.set_grad_enabled(False)
    return torch_model

In [6]:
model = get_model(9)

In [7]:
import torchvision.transforms as T
img_size = (254, 254)
train_transforms = get_transforms_for_predict()

In [8]:
def predict(frame):
    x_data = train_transforms(image=frame)
    x_data = x_data['image']
    prediction = model(x_data.unsqueeze(0))
    
    prediction = torch.round(prediction[0])
    
    SIZE = (prediction.shape[0], prediction.shape[1])
    frame = cv2.resize(frame, SIZE)
    
    final_image = viewer.mask_image(frame, prediction)
    
    final_image = cv2.resize(final_image, (1280, 720))

    return final_image
    

In [9]:
# Make a new directory
#os.mkdir("videos")

In [10]:
test_data_dir = '/mnt/heap/imges/road/testing/'

In [11]:
i_frame = 0
# Creating a VideoCapture object to read the video
project_video = "project_video.mp4"
original_video = cv2.VideoCapture(test_data_dir + project_video)
frame_width = int(original_video.get(3))
frame_height = int(original_video.get(4))
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
fps = 60
output = cv2.VideoWriter("videos/" + project_video, fourcc, fps, (frame_width,frame_height))

# Process Video
while(original_video.isOpened()):
    ret, frame = original_video.read()

    if ret == True:
        # Write the frame into the file 'output.avi'
        output.write(predict(frame))
        if i_frame == 10:
            break
        print(i_frame =

    else:
        break

# When everything done, release the video capture and video write objects
original_video.release()
output.release()

KeyboardInterrupt: 

In [ ]:
print('done')